# QuackOSM Command Line Interface

**QuackOSM** contains a CLI for users convenience. It is **not** installed by default when installed using `pip install quackosm`.

To include the CLI, **QuackOSM** has to be installed with additional group called `cli`: `pip install quackosm[cli]`.

CLI is based on the `typer` library and exposes almost all of the features implemented in the Python API.

After installation, the `QuackOSM` (or `quackosm`) command will be available in the shell.

Each command error returns a verbose description what went wrong.

## Basic usage

By default, the quackosm requires just the path to the `PBF` file. Without it, there will be an error.

In [ ]:
%config PlainTextFormatter.max_width = 120

In [ ]:
!stty size

In [ ]:
from shutil import get_terminal_size

get_terminal_size()

In [ ]:
import numpy as np
np.set_printoptions(linewidth=160)

In [ ]:
! QuackOSM

In [ ]:
! QuackOSM

Let's download a small extract and test basic usage.

In [ ]:
import urllib.request

monaco_pbf_url = "https://download.geofabrik.de/europe/monaco-latest.osm.pbf"
monaco_pbf_file = "monaco.osm.pbf"
urllib.request.urlretrieve(monaco_pbf_url, monaco_pbf_file)

In [ ]:
! QuackOSM monaco.osm.pbf

## Help command

If you want to learn more about available parameters to the QuackOSM, you can use the `--help` (or `-h`) parameter.

In [ ]:
! QuackOSM --help